In [3]:
#Reading the filtered parquet file
tw_df = spark.read.parquet("gs://msca-bdp-students-bucket/shared_data/suyashlakhani/tweets")

In [5]:
print(tw_df.count(),len(tw_df.columns))

91182863 22


In [7]:
import os
import shutil
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
pd.set_option("max_colwidth", 100)

from IPython.display import clear_output
clear_output(wait = False)

from google.cloud import storage

spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [8]:
tw_df_filt = tw_df.filter(
                          (lower(col('text')).contains("children"))\
                          | (lower(col('text')).contains("kindergraden"))\
                          | (lower(col('text')).contains("testing"))\
                          | (lower(col('text')).contains("grade"))\
                          | (lower(col('text')).contains("student"))\
                          | (lower(col('text')).contains("grades"))\
                          | (lower(col('text')).contains("education"))\
                          | (lower(col('text')).contains("k12"))\
                          | (lower(col('text')).contains("high school"))\
                          | (lower(col('text')).contains("college"))\
                          | (lower(col('text')).contains("elementary school"))\
                          | (lower(col('text')).contains("online education"))\
                          | (lower(col('text')).contains("math"))\
                          | (lower(col('text')).contains("science"))\
                          | (lower(col('text')).contains("k-12"))\
                          | (lower(col('text')).contains("middle school"))
                          | (lower(col('text')).contains("curriculum"))\
                          | (lower(col('text')).contains("learning"))\
                          | (lower(col('text')).contains("colleges"))\
                          | (lower(col('text')).contains("tuition"))\
                          | (lower(col('text')).contains("primary school"))
                          
                          & ~(lower(col('text')).contains("shot"))\
                          & ~(lower(col('text')).contains("kill"))\
                          & ~(lower(col('text')).contains("murder"))\
                          & ~(lower(col('text')).contains("murdered"))\
                          & ~(lower(col('text')).contains("shoot"))\
                          & ~(lower(col('text')).contains("killed"))\
                          & ~(lower(col('text')).contains("killing"))\
                          & ~(lower(col('text')).contains("attack"))\
                          & ~(lower(col('text')).contains("attacked"))\
                          & ~(lower(col('text')).contains("gun"))\
                          & ~(lower(col('text')).contains("weapon"))\
                          & ~(lower(col('text')).contains("sex"))\
                          & ~(lower(col('text')).contains("porn"))\
                          & ~(lower(col('text')).contains("tounament"))\
                          & ~(lower(col('text')).contains("weapons"))\
                          & ~(lower(col('text')).contains("massacre"))\
                          & ~(lower(col('text')).contains("dead"))
                          )

tw_df_filt.count()

22/12/04 19:17:47 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 27 for reason Container marked as failed: container_1670090078315_0006_01_000028 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-wwx8.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/12/04 19:17:47 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 28 for reason Container marked as failed: container_1670090078315_0006_01_000029 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-wwx8.c.msca-bdp-students.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
22/12/04 19:17:47 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 27 on hub-msca-bdp-dphub-students-suyashlakhani-sw-wwx8.c.msca-bdp-students.internal: Container marked as failed: container_1670090078315_0006_01_000028 on 

31523500

In [9]:
len(tw_df_filt.columns)

22

In [10]:
tw_df_filt.limit(5)

created_at,display_text_range,entities,favorite_count,favorited,filter_level,is_quote_status,lang,place,possibly_sensitive,quote_count,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user
Thu Jun 16 20:04:...,"[15, 140]","{[], null, [], [{...",0,false,low,false,en,null,false,0,0,0,,null,null,"<a href=""http://t...",@asamiamihoops Tr...,1655409847488,true,@asamiamihoops Tr...,"{false, Fri Feb 2..."
Thu Jun 16 20:04:...,null,"{[], null, [], []...",0,false,low,false,en,null,null,0,0,0,RT,itsJeffTiedrich,"{null, Thu Jun 16...","<a href=""http://t...",RT @itsJeffTiedri...,1655409847581,false,fun true fact: mo...,"{false, Mon Nov 0..."
Thu Jun 16 20:04:...,null,"{[], null, [], []...",0,false,low,false,en,null,null,0,0,0,RT,ReallyAmerican1,"{null, Thu Jun 16...","<a href=""https://...",RT @ReallyAmerica...,1655409848522,false,Who else agrees t...,"{false, Thu Jan 2..."
Thu Jun 16 20:04:...,null,"{[], null, [], []...",0,false,low,false,en,null,null,0,0,0,RT,DCUIntl,"{null, Thu Jun 16...","<a href=""http://t...",RT @DCUIntl: Cong...,1655409848823,false,Congrats to all o...,"{false, Mon Dec 2..."
Thu Jun 16 20:04:...,null,"{[], null, [], []...",0,false,low,false,en,null,null,0,0,0,RT,judgejules75,"{null, Thu Jun 16...","<a href=""http://t...",RT @judgejules75:...,1655409849853,false,Other orgs to end...,"{false, Mon Jun 1..."


In [24]:
#Extracting important information from the columns and selecting only the required columns required for analysis
data_final = tw_df_filt.select([tw_df_filt.created_at,
                         tw_df_filt.place.country.alias("country"),
                         tw_df_filt.user, 
                         tw_df_filt.user['id'].alias("user_id"),
                         tw_df_filt.user['name'].alias("user_name"), 
                         tw_df_filt.user['description'].alias("user_description"), 
                         tw_df_filt.user['followers_count'].alias("followers_count"), 
                         tw_df_filt.retweeted_status.quote_count.alias("quote_count"),
                         tw_df_filt.retweeted_status,
                         tw_df_filt.retweeted_status.retweet_count.alias("retweet_count"),
                         tw_df_filt.retweeted_status.favorite_count.alias("favorite_count"),
                         tw_df_filt.text, 
                         tw_df_filt.user.verified.alias("verified_user")])

data_final.limit(5).toPandas()

,created_at,country,user,user_id,user_name,user_description,followers_count,quote_count,retweeted_status,retweet_count,favorite_count,text,verified_user
0,Sat Sep 17 17:08:22 +0000 2022,None,"(False, Sat May 22 06:33:12 +0000 2021, True, False, None, 3644, 198, 546, False, 13959908890848...",1395990889084817410,1158 APLC Jaskarn Singh,None,198,0.0,"(None, Sat Sep 17 16:00:28 +0000 2022, None, ([], None, [], [Row(display_url='twitter.com/i/web/...",10.0,8.0,RT @ArshWariana: Once there used to be good quality govt colleges in Punjab.This statement is go...,False
1,Sat Sep 17 17:08:23 +0000 2022,None,"(False, Tue Jan 11 16:55:28 +0000 2022, True, False, None, 34421, 50, 118, False, 14809463720775...",1480946372077522950,Suren,None,50,21.0,"(None, Fri Sep 16 10:15:57 +0000 2022, [0, 140], ([Row(indices=[0, 9], text='BREAKING')], None, ...",1156.0,5591.0,RT @republic: #BREAKING | My heart goes out to girls who should have been in college but are han...,False
2,Sat Sep 17 17:08:23 +0000 2022,None,"(False, Thu Aug 05 18:44:26 +0000 2021, True, False, (She/her). Pansexual. Postgrad. Research in...",1423354181075673091,Debadrita Saha 🌈,(She/her). Pansexual. Postgrad. Research int. at @decoloniszing. Former research int. @MCRG_CRG....,430,73.0,"(None, Fri Sep 16 15:41:50 +0000 2022, None, ([], None, [], [], []), None, None, 1335, False, lo...",112.0,1335.0,RT @StefanoBloch: Professors. You assign too much reading. It doesn't help. Assign less so stude...,False
3,Sat Sep 17 17:08:23 +0000 2022,None,"(False, Sat Nov 19 19:06:40 +0000 2011, False, False, SPORTS, 0, 151, 16, False, 416481014, 4164...",416481014,HS SPORTS,SPORTS,151,NaN,None,NaN,NaN,Hamady vs Madison Academy | Today's Michigan High School Football Live Stream\nWatch live Broadc...,False
4,Sat Sep 17 17:08:24 +0000 2022,None,"(False, Fri Dec 30 19:20:02 +0000 2011, True, False, 💜, 16721, 108, 254, False, 450871097, 45087...",450871097,Annabel bacon,💜,108,341.0,"(None, Sat Sep 17 01:51:48 +0000 2022, None, ([], None, [], [Row(display_url='twitter.com/i/web/...",6556.0,21797.0,RT @drsimonegold: BREAKING: A senior high school student at Holmes County High School in Florida...,False


In [15]:
print(data_final.count(),len(data_final.columns))

22/12/04 20:51:01 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670090078315_0006_01_000036 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-3dnh.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 20:51:01.210]Container killed on request. Exit code is 143
[2022-12-04 20:51:01.211]Container exited with a non-zero exit code 143. 
[2022-12-04 20:51:01.211]Killed by external signal
.
22/12/04 20:51:01 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 35 on hub-msca-bdp-dphub-students-suyashlakhani-sw-3dnh.c.msca-bdp-students.internal: Container from a bad node: container_1670090078315_0006_01_000036 on host: hub-msca-bdp-dphub-students-suyashlakhani-sw-3dnh.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-04 20:51:01.210]Container killed on request. Exit code is 143
[2022-12-04 20:51:01.211]Container exited with a non-zero exit code 143. 
[2022-12-04 20:51:01.211]Killed by external 

31523500 13


In [16]:
df_null = data_final.select([F.count(F.when(data_final[c].isNull(), c)).alias(c) for c in data_final.columns]).toPandas()
df_null

,created_at,country,user,user_id,user_name,user_description,followers_count,quote_count,retweeted_status,retweet_count,favorite_count,text,verified_user
0,0,31225884,0,0,0,4601793,0,10374152,10374152,10374152,10374152,0,0


In [25]:
data_final = data_final.na.fill(value=0,subset=["retweet_count", "quote_count", "favorite_count"])
data_final = data_final.na.drop(subset = ['user_description'])
data_final = data_final.drop(col("country"))

--> I handled the missing values of the important columns by either dropping the values(if less) or filling it with 0. 

In [23]:
data_final.limit(5).toPandas()

,user_location
0,None
1,Central Ohio
2,None
3,"Baltimore, MD, USA"
4,San Francisco


In [28]:
data_final = data_final.withColumn("user_location",data_final.user.location)                       
data_final.limit(5).toPandas()

,created_at,user,user_id,user_name,user_description,followers_count,quote_count,retweeted_status,retweet_count,favorite_count,text,verified_user,user_location
0,Sat Sep 17 17:08:23 +0000 2022,"(False, Thu Aug 05 18:44:26 +0000 2021, True, False, (She/her). Pansexual. Postgrad. Research in...",1423354181075673091,Debadrita Saha 🌈,(She/her). Pansexual. Postgrad. Research int. at @decoloniszing. Former research int. @MCRG_CRG....,430,73,"(None, Fri Sep 16 15:41:50 +0000 2022, None, ([], None, [], [], []), None, None, 1335, False, lo...",112,1335,RT @StefanoBloch: Professors. You assign too much reading. It doesn't help. Assign less so stude...,False,None
1,Sat Sep 17 17:08:23 +0000 2022,"(False, Sat Nov 19 19:06:40 +0000 2011, False, False, SPORTS, 0, 151, 16, False, 416481014, 4164...",416481014,HS SPORTS,SPORTS,151,0,None,0,0,Hamady vs Madison Academy | Today's Michigan High School Football Live Stream\nWatch live Broadc...,False,Central Ohio
2,Sat Sep 17 17:08:24 +0000 2022,"(False, Fri Dec 30 19:20:02 +0000 2011, True, False, 💜, 16721, 108, 254, False, 450871097, 45087...",450871097,Annabel bacon,💜,108,341,"(None, Sat Sep 17 01:51:48 +0000 2022, None, ([], None, [], [Row(display_url='twitter.com/i/web/...",6556,21797,RT @drsimonegold: BREAKING: A senior high school student at Holmes County High School in Florida...,False,None
3,Sat Sep 17 17:08:24 +0000 2022,"(False, Thu Mar 12 10:36:22 +0000 2009, True, False, health care admin, happily married, 5 grand...",23929337,Ira Greene 🌊♻️🌐☮️🌈🐰♋✡️,"health care admin, happily married, 5 grandkids, progressive ex-hippie Deadhead still wonder wha...",2458,2,"(None, Sat Sep 17 16:32:08 +0000 2022, None, ([], None, [], [], []), None, None, 59, False, low,...",34,59,RT @ungerbn103: Republicans will fund Abortion Bounty Hunters but they won't fund free School Lu...,False,"Baltimore, MD, USA"
4,Sat Sep 17 17:08:24 +0000 2022,"(False, Wed Feb 11 01:06:47 +0000 2009, False, False, Husband, Dog-Dude, Self-deprecating barkee...",20562088,BrianCass,"Husband, Dog-Dude, Self-deprecating barkeep of @noirloungesf SFBay. @warriors @sfgiants @49ers",399,0,None,0,0,@stoolpresidente Has a college QB ever had such a Cliff dive than Rattler. Wasn’t he in the con...,False,San Francisco


In [34]:
data_final.write.format("parquet").mode('overwrite').save('gs://msca-bdp-students-bucket/shared_data/suyashlakhani/tweets2')

-->This is the final filtered dataframe containing the education related tweets and important columns for analysis